# 食品制造问题 II

## 目标和预备知识

在这个例子中,你需要解决与"食品制造问题 I"相同的问题,但是添加了额外的约束条件。这些约束条件将问题从线性规划(LP)问题转变为混合整数规划(MIP)问题,使其更难求解。

关于这种类型模型的更多信息可以在 H. P. Williams 所著《数学规划建模》第五版的示例 #2(第 255 页和第 299-300 页)中找到。

这是一个中级建模示例,我们假设你了解 Python 并且熟悉 Gurobi Python API。此外,你还需要具备一些数学优化建模的知识。

**下载代码库** <br />
你可以通过点击[此处](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一个制造商需要加工几种原油并将它们混合在一起生产可以销售的食品产品。所需的原油可以分为两类:


| 类别 | 油 |
| ------------- |-------------|
| 植物油:| VEG 1<br>VEG 2 |
| 非植物油: | OIL 1<br>OIL 2<br>OIL 3 |


制造商可以选择为当月购买原油和/或在期货市场上购买供后续月份交付的原油。以下是即期交货和期货市场的价格(美元/吨):

| 月份 | VEG 1 | VEG 2 | OIL 1 | OIL 2 | OIL 3|
| ------------- |-------------| -------------| -------------| -------------| -------------| 
| 一月 | 110 | 120 | 130 | 110 | 115|
| 二月 |130 | 130 | 110 | 90| 115|
| 三月 |110 | 140 | 130 | 100 | 95|
| 四月 |120 | 110 | 120 | 120 | 125|
| 五月 | 100 | 120 | 150 | 110 | 105|
| 六月 | 90 | 100 | 140 | 80| 135 |

还必须考虑一些其他因素。这些包括:

1. 最终食品产品售价为 150 美元/吨。
2. 每种类别的油(植物油和非植物油)需要在不同的生产线上精炼。
3. 精炼能力有限,在任何给定月份最多可以精炼 200 吨植物油和 250 吨非植物油。
4. 此外,在精炼过程中没有浪费,因此精炼的原油总量将等于可用的精炼油量。
5. 精炼油的成本可以忽略不计。

除了上述精炼限制外,还有存储原油以供将来使用的限制,并且存储每吨油都有相关成本。限制是每种原油 1,000 吨,存储成本为每吨每月 5 美元。制造商不能储存生产的食品产品或精炼油。

最终食品产品在给定硬度量表上的硬度必须在 3 到 6 之间。就模型而言,硬度呈线性混合,每种原油的硬度为:

|油 | 硬度|
| ------------- |-------------|
|VEG 1 | 8.8|
|VEG 2 | 6.1|
|OIL 1 | 2.0|
|OIL 2 | 4.2|
|OIL 3| 5.0|

在一月初,每种原油的库存量为 500 吨。就模型而言,这也应该是六月末原油库存的水平。

这个版本的食品制造问题在第一个版本的基础上增加了以下额外约束:

- 条件 1: 如果在某个月份使用某种油,则最小使用量必须为 20 吨。
- 条件 2: 一个月内最多使用三种油。
- 条件 3: 在给定月份使用 VEG1 或 VEG2 需要在同一月份使用 OIL3。


给定上述信息,应该做出哪些月度采购和制造决策以最大化利润?

---
## 模型构建

### 集合和索引

$t \in \text{Months}=\{\text{Jan},\text{Feb},\text{Mar},\text{Apr},\text{May},\text{Jun}\}$: 月份集合。

$V=\{\text{VEG1},\text{VEG2}\}$: 植物油集合。

$N=\{\text{OIL1},\text{OIL2},\text{OIL3}\}$: 非植物油集合。

$o \in \text{Oils} = V \cup N$: 油类集合。

### 参数

$\text{price} \in \mathbb{R}^+$: 最终产品的销售价格。

$\text{init_store} \in \mathbb{R}^+$: 初始库存量(吨)。

$\text{target_store} \in \mathbb{R}^+$: 目标库存量(吨)。

$\text{holding_cost} \in \mathbb{R}^+$: 每吨油每月的库存成本(美元/吨/月)。

$\text{min_consume} \in \mathbb{R}^+$: 某个月份使用某种油的最小吨数。

$\text{veg_cap} \in \mathbb{R}^+$: 植物油精炼产能(吨)。

$\text{oil_cap} \in \mathbb{R}^+$: 非植物油精炼产能(吨)。

$\text{min_hardness} \in \mathbb{R}^+$: 最终产品允许的最低硬度。

$\text{max_hardness} \in \mathbb{R}^+$: 最终产品允许的最高硬度。

$\text{hardness}_o \in \mathbb{R}^+$: 油 $o$ 的硬度。

$\text{max_ingredients} \in \mathbb{N}$: 某个月份最多使用的油种类数。

$\text{cost}_{t,o} \in \mathbb{R}^+$: 月份 $t$ 时油 $o$ 的预计购买价格。


### 决策变量

$\text{produce}_t \in \mathbb{R}^+$: 月份 $t$ 生产的食品吨数。

$\text{buy}_{t,o} \in \mathbb{R}^+$: 月份 $t$ 购买的油 $o$ 的吨数。

$\text{consume}_{t,o} \in \mathbb{R}^+$: 月份 $t$ 使用的油 $o$ 的吨数。

$\text{store}_{t,o} \in \mathbb{R}^+$: 月份 $t$ 储存的油 $o$ 的吨数。

$\text{use}_{t,o} \in \{0,1\}$: 如果月份 $t$ 使用油 $o$ 则为 1,否则为 0。


### 目标函数

- **利润**: 最大化规划期内的总利润(美元)。

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\text{price}*\text{produce}_t - \sum_{t \in \text{Months}}\sum_{o \in \text{Oils}}(\text{cost}_{t,o}*\text{consume}_{t,o} + \text{holding_cost}*\text{store}_{t,o})
\tag{0}
\end{equation}

### 约束条件

- **初始平衡:** 一月购买的油 $o$ 吨数和之前储存的量之和应等于该月消耗和储存的量之和。

\begin{equation}
\text{init store} + \text{buy}_{Jan,o} = \text{consume}_{Jan,o} + \text{store}_{Jan,o} \quad \forall o \in \text{Oils}
\tag{1}
\end{equation}

- **平衡:** 月份 $t$ 购买的油 $o$ 吨数和之前储存的量之和应等于该月消耗和储存的量之和。

\begin{equation}
\text{store}_{t-1,o} + \text{buy}_{t,o} = \text{consume}_{t,o} + \text{store}_{t,o} \quad \forall (t,o) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Oils}
\tag{2}
\end{equation}

- **库存目标**: 规划期末储存的油 $o$ 吨数应达到目标值。

\begin{equation}
\text{store}_{Jun,o} = \text{target_store} \quad \forall o \in \text{Oils}
\tag{3}
\end{equation}

- **精炼能力**: 月份 $t$ 消耗的油 $o$ 总吨数不能超过精炼能力。

\begin{equation}
\sum_{o \in V}\text{consume}_{t,o} \leq \text{veg_cap} \quad \forall t \in \text{Months}
\tag{4.1}
\end{equation}

\begin{equation}
\sum_{o \in N}\text{consume}_{t,o} \leq \text{oil_cap} \quad \forall t \in \text{Months}
\tag{4.2}
\end{equation}

- **硬度**: 月份 $t$ 生产的食品的硬度值应在容许范围内。

\begin{equation}
\text{min_hardness}*\text{produce}_t \leq \sum_{o \in \text{Oils}} \text{hardness}_o*\text{consume}_{t,o} \leq \text{max_hardness}*\text{produce}_t \quad \forall t \in \text{Months}
\tag{5}
\end{equation}

- **质量守恒**: 月份 $t$ 消耗的油总吨数应等于该月生产的食品吨数。

\begin{equation}
\sum_{o \in \text{Oils}}\text{consume}_{t,o} = \text{produce}_t \quad \forall t \in \text{Months}
\tag{6}
\end{equation}

- **消耗范围**: 如果我们决定在月份 $t$ 使用油 $o$,则消耗量应在 20 吨到其类型的精炼能力之间。

\begin{equation}
\text{min_consume}*\text{use}_{t,o} \leq \text{consume}_{t,o} \leq \text{veg_cap}*\text{use}_{t,o} \quad \forall (t,o) \in V \times \text{Months}
\tag{7.1}
\end{equation}

\begin{equation}
\text{min_consume}*\text{use}_{t,o} \leq \text{consume}_{t,o} \leq \text{oil_cap}*\text{use}_{t,o} \quad \forall (t,o) \in N \times \text{Months}
\tag{7.2}
\end{equation}

- **配方**: 月份 $t$ 使用的油种类数最多为三种。

\begin{equation}
\sum_{o \in \text{Oils}}\text{use}_{t,o} \leq \text{max_ingredients} \quad \forall t \in \text{Months}
\tag{8}
\end{equation}

- **如果-则约束**: 如果在月份 $t$ 使用 VEG1 或 VEG2,则必须在该月使用 OIL3。

\begin{equation}
\text{use}_{t,\text{VEG1}} \leq \text{use}_{t,\text{OIL3}} \quad \forall t \in \text{Months}
\tag{9.1}
\end{equation}

\begin{equation}
\text{use}_{t,\text{VEG2}} \leq \text{use}_{t,\text{OIL3}} \quad \forall t \in \text{Months}
\tag{9.2}
\end{equation}

---
## Python 实现
我们导入 Gurobi Python 模块和其他 Python 库。


In [ ]:
# %pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7 & Gurobi 9

## 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 参数

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]

cost = {
    ('Jan', 'VEG1'): 110,
    ('Jan', 'VEG2'): 120,
    ('Jan', 'OIL1'): 130,
    ('Jan', 'OIL2'): 110,
    ('Jan', 'OIL3'): 115,
    ('Feb', 'VEG1'): 130,
    ('Feb', 'VEG2'): 130,
    ('Feb', 'OIL1'): 110,
    ('Feb', 'OIL2'): 90,
    ('Feb', 'OIL3'): 115,
    ('Mar', 'VEG1'): 110,
    ('Mar', 'VEG2'): 140,
    ('Mar', 'OIL1'): 130,
    ('Mar', 'OIL2'): 100,
    ('Mar', 'OIL3'): 95,
    ('Apr', 'VEG1'): 120,
    ('Apr', 'VEG2'): 110,
    ('Apr', 'OIL1'): 120,
    ('Apr', 'OIL2'): 120,
    ('Apr', 'OIL3'): 125,
    ('May', 'VEG1'): 100,
    ('May', 'VEG2'): 120,
    ('May', 'OIL1'): 150,
    ('May', 'OIL2'): 110,
    ('May', 'OIL3'): 105,
    ('Jun', 'VEG1'): 90,
    ('Jun', 'VEG2'): 100,
    ('Jun', 'OIL1'): 140,
    ('Jun', 'OIL2'): 80,
    ('Jun', 'OIL3'): 135
}


hardness = {"VEG1": 8.8, "VEG2": 6.1, "OIL1": 2.0, "OIL2": 4.2, "OIL3": 5.0}

price = 150
init_store = 500
veg_cap = 200
oil_cap = 250

min_hardness = 3
max_hardness = 6
max_ingredients = 3
holding_cost = 5
min_consume = 20

## 模型部署

对于每个时期,我们创建一个变量来考虑生产的食品的价值。对于每种产品(五种油)和每个时期,我们将创建变量来表示购买量、使用量和储存量。

对于每个时期和每种产品,我们需要一个二进制变量,用于表示该产品在当前时期是否被使用。

In [ ]:
food = gp.Model('Food Manufacture II')
# 每个时期生产的食品数量
produce = food.addVars(months, name="Food")
# 每个时期购买的每种产品数量
buy = food.addVars(months, oils, name = "Buy")
# 每个时期使用的每种产品数量
consume = food.addVars(months, oils, name = "Consume")
# 每个时期储存的每种产品数量
store = food.addVars(months, oils, name = "Store")
# 二进制变量=1,如果消耗>0
use = food.addVars(months, oils, vtype=GRB.BINARY, name = "Use")

Using license file c:\gurobi\gurobi.lic


接下来,我们插入约束条件。平衡约束确保上一期储存的油量加上购买的量等于当前期使用的量加上储存的量(对于每种油)。

In [ ]:
#1. 初始平衡
Balance0 = food.addConstrs((init_store + buy[months[0], oil]
                 == consume[months[0], oil] + store[months[0], oil]
                 for oil in oils), "Initial_Balance")

#2. 平衡
Balance = food.addConstrs((store[months[months.index(month)-1], oil] + buy[month, oil]
                 == consume[month, oil] + store[month, oil]
                 for oil in oils for month in months if month != months[0]), "Balance")

库存目标约束强制要求在最后一期结束时,每种油的库存量要达到初始量。问题描述要求储存量要与开始时一样多。

In [ ]:
#3. 库存目标
TargetInv = food.addConstrs((store[months[-1], oil] == init_store for oil in oils), "End_Balance")

能力约束限制每期可以加工的植物油和非植物油的数量。每月只能将 200 吨植物油和 250 吨非植物油加工成最终产品。

In [ ]:
#4.1 植物油产能
VegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "VEG" in oil)
                 <= veg_cap for month in months), "Capacity_Veg")

#4.2 非植物油产能
NonVegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "OIL" in oil)
                 <= oil_cap for month in months), "Capacity_Oil")

硬度约束限制最终产品的硬度,需要保持在 3 到 6 之间。每种油都有一个特定的硬度。最终产品可能由不同的油组成。最终产品的硬度由每种成分的硬度乘以其在最终产品中的份额来衡量。假设硬度呈线性混合。

In [ ]:
#5. 硬度
HardnessMin = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 >= min_hardness*produce[month] for month in months), "Hardness_lower")
HardnessMax = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 <= max_hardness*produce[month] for month in months), "Hardness_upper")

质量守恒约束确保在每个期间使用的产品量等于在该期间生产的食品量。这确保所有使用的油都被加工成最终产品(食品)。

In [ ]:
#6. 质量守恒
MassConservation = food.addConstrs((consume.sum(month) == produce[month] for month in months), "Mass_conservation")

条件 1 约束强制要求如果在任何期间使用任何产品,则至少要使用 20 吨。它们还强制要求如果且仅当用于同一产品和同一月份的连续变量非零时,每种产品和每月的二进制变量设置为 1。二进制变量称为指示变量,因为它与连续变量相关联并指示它是否非零。

将条件 1 表示为纯 MIP 约束集相对简单。让我们看看如何使用 Gurobi 的通用约束(从 7.0 版本开始)来建模这个集合:

In [ ]:
#7.1 & 7.2 消耗范围 - 使用Gurobi的通用约束
for month in months:
    for oil in oils:
        food.addGenConstrIndicator(use[month, oil], 0,
                                   consume[month, oil] == 0,
                                   name="Lower_bound_{}_{}".format(month, oil))
        food.addGenConstrIndicator(use[month, oil], 1,
                                   consume[month, oil] >=  min_consume,
                                   name="Upper_bound_{}_{}".format(month, oil))

条件 2 约束确保每个最终产品最多由三种成分组成。

In [ ]:
#8. 配方
condition2 = food.addConstrs((use.sum(month) <= max_ingredients for month in months),"Recipe")

条件 3 约束确保如果使用植物油一或植物油二,则必须使用油三。我们将再次使用 Gurobi 的通用约束:

In [ ]:
#9.1 & 9.2 如果-则约束
for month in months:
    food.addGenConstrIndicator(use[month, "VEG1"], 1,
                               use[month, "OIL3"] == 1,
                               name = "If_then_a_{}".format(month))
    food.addGenConstrIndicator(use[month, "VEG2"], 1,
                               use[month, "OIL3"] == 1,
                               name = "If_then_b_{}".format(month))

目标是最大化公司的利润。这通过收入减去购买和储存所购产品(成分)的成本来计算。

In [ ]:
#0. 目标函数
obj = price*produce.sum() - buy.prod(cost) - holding_cost*store.sum()
food.setObjective(obj, GRB.MAXIMIZE) # 最大化利润

接下来,我们开始优化,Gurobi 找到最优解。

In [13]:
food.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 71 rows, 126 columns and 288 nonzeros
Model fingerprint: 0x9d193fb2
Model has 72 general constraints
Variable types: 96 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 5e+02]
Presolve added 24 rows and 0 columns
Presolve removed 0 rows and 66 columns
Presolve time: 0.01s
Presolved: 95 rows, 60 columns, 272 nonzeros
Variable types: 35 continuous, 25 integer (25 binary)
Found heuristic solution: objective 34650.000000

Root relaxation: objective 1.056500e+05, 57 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 105650.000    0    6 34650.0000 105650.000   205%   

---
## 分析

在最初设计时,这个模型相对难以求解(参见食品制造问题 I)。这个计划产生的利润(销售收入减去原油成本)为 $100,278.7 美元。存在其他同样好的解决方案。

### 采购计划

该计划定义了在规划期内需要购买的植物油(VEG)和非植物油(OIL)的数量。例如,在六月需要购买 480.4 吨 VEG1 型植物油。

In [14]:
rows = months.copy()
columns = oils.copy()
purchase_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in buy.keys():
    if (abs(buy[month, oil].x) > 1e-6):
        purchase_plan.loc[month, oil] = np.round(buy[month, oil].x, 1)
purchase_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,0.0,0.0,0.0,0.0
Feb,0.0,0.0,0.0,190.0,0.0
Mar,0.0,0.0,0.0,0.0,540.0
Apr,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,0.0,0.0,40.0
Jun,480.4,629.6,0.0,730.0,0.0


### 月度消耗

该计划确定了规划期内消耗的植物油(VEG)和非植物油(OIL)的数量。例如,在一月消耗 114.8 吨 VEG2 型植物油。

In [15]:
rows = months.copy()
columns = oils.copy()
reqs = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in consume.keys():
    if (abs(consume[month, oil].x) > 1e-6):
        reqs.loc[month, oil] = np.round(consume[month, oil].x, 1)
reqs

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,200.0,0.0,230.0,20.0
Feb,85.2,114.8,0.0,0.0,250.0
Mar,85.2,114.8,0.0,0.0,250.0
Apr,155.0,0.0,0.0,230.0,20.0
May,155.0,0.0,0.0,230.0,20.0
Jun,0.0,200.0,0.0,230.0,20.0


### 库存计划

该计划反映了规划期内每个期末植物油(VEG)和非植物油(OIL)的库存量。例如,在二月底我们有 500 吨 OIL1 型非植物油。

In [16]:
rows = months.copy()
columns = oils.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in store.keys():
    if (abs(store[month, oil].x) > 1e-6):
        store_plan.loc[month, oil] = np.round(store[month, oil].x, 1)
store_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,500.0,300.0,500.0,270.0,480.0
Feb,414.8,185.2,500.0,460.0,230.0
Mar,329.6,70.4,500.0,460.0,520.0
Apr,174.6,70.4,500.0,230.0,500.0
May,19.6,70.4,500.0,0.0,520.0
Jun,500.0,500.0,500.0,500.0,500.0


注意:如果你想将解决方案写入文件,而不是打印到终端,可以使用 model.write() 命令。示例实现为:

`food.write("food-manufacture-2-output.sol")`

---
## 参考文献

H. Paul Williams,《数学规划建模》第五版。

版权所有 © 2020 Gurobi Optimization, LLC